# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from memexplatform_obsidian.commons import config, MountPaths
from memexplatform.ui.structure import ifhtmx
from memexplatform_obsidian.mdmanager import ObsidianHTMLRenderer, get_obsidianmd_ast
from mistletoe import Document
import urllib

In [ ]:
#| export
def create_app():
    hdrs = Theme.neutral.headers(highlightjs=True)
    script = Script(src="https://unpkg.com/@superset-ui/embedded-sdk")
    newhdrs = hdrs  + [script]
    mount_routes = []

    app = FastHTML(hdrs=newhdrs, live=True, exts='ws', routes=mount_routes)
    # app.add_middleware(SessionSyncMiddleware)
    rt = app.route
    return app, rt

app, rt = create_app()

In [ ]:
MountPaths.gradio_url.value

'/obsidian_gradio'

In [ ]:
#| export
@rt
def index(request:Request):

    # listteam_comp = Container(Div('Team'), hx_get=f'/portal/list_team', hx_target='#listteam', id='listteam', hx_swap='innerHTML',hx_vals=json.dumps({"folder": str(config.NBPATH)}),  hx_trigger='load')
    comp = Div("Obsidian Companion")
    
    return ifhtmx(request, comp)

In [ ]:
#| export
@app.get('/open')
def app_route(request:Request, file: str = ""):
    vault_path = config.OBSIDIAN_VAULT
    file_path = vault_path / urllib.parse.unquote(file)

    if not file_path.exists():
        return Div(f"Path not found: {file_path}")

    # If it's a directory → list contents
    if file_path.is_dir():
        items = []
        for p in sorted(file_path.iterdir()):
            rel_path = p.relative_to(vault_path)
            href = MountPaths.open.to(file=rel_path)
            items.append(Li(A(p.name, href=href)))
        return ifhtmx(request,Div(
            f"Listing for {file_path}:", 
            Ul(*items)
        ))

    # If it's a file → show content (assuming text)
    if file_path.is_file():
        doc = None
        try:
            text = file_path.read_text(encoding="utf-8")
        except Exception:
            text = "[Unable to read file]"

        html = None
        with ObsidianHTMLRenderer() as renderer:
            doc = get_obsidianmd_ast(text)
            html = renderer.render(doc)
            print(html)
        return ifhtmx(request, Div(
            f"File: {file_path}",
            NotStr(apply_classes(html))
        ))



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()